In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'e:\\MLs\\MLOPS Project\\SimpleMLopsProject'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [8]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 scherma_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(scherma_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_model_train_config(self) -> ModelTrainConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_train_config = ModelTrainConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        
        return model_train_config
    
    

In [12]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from src import logger
import os
import joblib

In [13]:
class ModelTrainer:
    def __init__(self, config: ModelTrainConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop(columns=[self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_x = test_data.drop(columns=[self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        model = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=42
        )
        
        model.fit(train_x, train_y)
        
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [14]:
try:
    config = ConfigurationManager()
    model_train_config = config.get_model_train_config()
    model_trainer = ModelTrainer(config=model_train_config)
    model_trainer.train()
except Exception as e:
    logger.exception(f"Error occurred during model training: {e}")
    raise e

2025-07-28 21:33:59,057 - SimpleMLopsLogger - INFO - YAML file config\config.yaml read successfully.
2025-07-28 21:33:59,062 - SimpleMLopsLogger - INFO - YAML file params.yaml read successfully.
2025-07-28 21:33:59,071 - SimpleMLopsLogger - INFO - YAML file schema.yaml read successfully.
2025-07-28 21:33:59,075 - SimpleMLopsLogger - INFO - Created directory: artifacts
2025-07-28 21:33:59,076 - SimpleMLopsLogger - INFO - Created directory: artifacts/model_trainer
